In [3]:
import mpmath as mp
import pandas as pd
from functools import lru_cache

mp.dps = 60  

# P polynomial is polynomial where coefficients are stirling numbers of second kind
def stirling2(n, k, memo={}):
    if (n, k) in memo:
        return memo[(n, k)]
    if n == k == 0:
        memo[(n, k)] = 1
    elif n == 0 or k == 0:
        memo[(n, k)] = 0
    else:
        memo[(n, k)] = k * stirling2(n - 1, k) + stirling2(n - 1, k - 1)
    return memo[(n, k)]

@lru_cache(maxsize=None)
def P_coeffs(p):
    return [((-1) ** j) * stirling2(p + 1, j + 1) for j in range(p + 1)]

def P_poly(p, u):
    res = mp.mpf("0")
    for coeff in reversed(P_coeffs(p)):
        res = res * u + coeff
    return res

def B_p(p):
    integrand = lambda u: mp.e**(-u) * abs(P_poly(p, u))
    return mp.quad(integrand, [0, mp.inf])

def zeta_factor(p):
    return 2 * mp.zeta(p) / (2 * mp.pi) ** p  # 2 ζ(p) / (2π)^p

def non_k_compute(p):
    return zeta_factor(p) * B_p(p)

def k_compute(k, p):
    return (mp.log(k) ** (p - 1)) 

def C_p(p, k):
    return k_compute(k,p) * non_k_compute(p)


In [5]:
from dataclasses import dataclass
@dataclass
class InformationAboutIntegral:
    p: int
    b_p: float
    factor: float
    non_k_compute: float

results: list[InformationAboutIntegral] = []

pd.set_option('display.float_format', '{:.5f}'.format)
begin_p=2
end_p=100
rows = []
for p in range(begin_p, end_p+1, 2):
    results.append(
        InformationAboutIntegral(
            p=p,
            b_p=B_p(p),
            factor=zeta_factor(p),
            non_k_compute=non_k_compute(p)
        )
    )
    rows.append({
        "p": p,
        "B_p": B_p(p),
        "factor": zeta_factor(p),
        "non_k_compute": non_k_compute(p),
    })

df = pd.DataFrame(rows)
# 1) zwykły print – najmniej zależności
from tabulate import tabulate
print(tabulate(df, headers="keys", tablefmt="github", floatfmt=".6e"))

|    |   p |           B_p |       factor |   non_k_compute |
|----|-----|---------------|--------------|-----------------|
|  0 |   2 | 9.404206e-01  | 8.333333e-02 |    7.836839e-02 |
|  1 |   4 | 4.512429e+00  | 1.388889e-03 |    6.267263e-03 |
|  2 |   6 | 5.436321e+01  | 3.306878e-05 |    1.797725e-03 |
|  3 |   8 | 1.223204e+03  | 8.267196e-07 |    1.011247e-03 |
|  4 |  10 | 4.448425e+04  | 2.087676e-08 |    9.286868e-04 |
|  5 |  12 | 2.385600e+06  | 5.284190e-10 |    1.260596e-03 |
|  6 |  14 | 1.764328e+08  | 1.338254e-11 |    2.361118e-03 |
|  7 |  16 | 1.711968e+10  | 3.389680e-13 |    5.803024e-03 |
|  8 |  18 | 2.112054e+12  | 8.586062e-15 |    1.813423e-02 |
|  9 |  20 | 3.303247e+14  | 2.174869e-16 |    7.184128e-02 |
| 10 |  22 | 6.011441e+16  | 5.509003e-18 |    3.311705e-01 |
| 11 |  24 | 1.333706e+19  | 1.395446e-19 |    1.861116e+00 |
| 12 |  26 | 3.485967e+21  | 3.534707e-21 |    1.232187e+01 |
| 13 |  28 | 1.092238e+24  | 8.953517e-23 |    9.779374e+01 |
| 14 |  

In [9]:
import numpy as np

def find_upper_limit_for_given_k(k: float):
    best_p = -1
    best_upper_limit = np.inf

    for res in results:
        kcompute = k_compute(k, res.p)
        possible_upper_limit = res.non_k_compute * kcompute
        if possible_upper_limit < best_upper_limit:
            # print(res.p, possible_upper_limit)
            best_upper_limit = possible_upper_limit
            best_p = res.p
    
    return (best_p, best_upper_limit)


In [7]:
find_upper_limit_for_given_k(2)

(14, mpf('2.0130231410180716e-5'))

In [8]:
for kk in np.arange(11, 41, 1):
    k = kk/10
    (p, bul) = find_upper_limit_for_given_k(k)
    print(f"Dla {k=}, Reszta R jest najmniejsza dla {p=}, i wynosi {float(bul):.10f}")

Dla k=np.float64(1.1), Reszta R jest najmniejsza dla p=64, i wynosi 0.0000000000
Dla k=np.float64(1.2), Reszta R jest najmniejsza dla p=60, i wynosi 0.0000000000
Dla k=np.float64(1.3), Reszta R jest najmniejsza dla p=36, i wynosi 0.0000000000
Dla k=np.float64(1.4), Reszta R jest najmniejsza dla p=28, i wynosi 0.0000000000
Dla k=np.float64(1.5), Reszta R jest najmniejsza dla p=24, i wynosi 0.0000000018
Dla k=np.float64(1.6), Reszta R jest najmniejsza dla p=20, i wynosi 0.0000000423
Dla k=np.float64(1.7), Reszta R jest najmniejsza dla p=18, i wynosi 0.0000003801
Dla k=np.float64(1.8), Reszta R jest najmniejsza dla p=16, i wynosi 0.0000020042
Dla k=np.float64(1.9), Reszta R jest najmniejsza dla p=14, i wynosi 0.0000074095
Dla k=np.float64(2.0), Reszta R jest najmniejsza dla p=14, i wynosi 0.0000201302
Dla k=np.float64(2.1), Reszta R jest najmniejsza dla p=12, i wynosi 0.0000472733
Dla k=np.float64(2.2), Reszta R jest najmniejsza dla p=12, i wynosi 0.0000922861
Dla k=np.float64(2.3), Reszt